# FRIDA: A Collaborative Robot Painter with a Differentiable, Real2Sim2Real Planning Environment

#### [Peter Schaldenbrand](https://pschaldenbrand.github.io/#about.html), [Jean Oh](https://www.cs.cmu.edu/~./jeanoh/), [Jim McCann](http://www.cs.cmu.edu/~jmccann/)
#### The Robotics Institute, Carnegie Mellon University

There's code for multiple different painting tasks below. Feel free to take a look at the command line arguments to change parameters such as the number of brush strokes used or number of optimization iterations.

# Installation

In [1]:
%load_ext tensorboard

In [ ]:
!git clone https://github.com/pschaldenbrand/Frida.git

In [3]:
cd /content/Frida/src

/content/Frida/src


In [4]:
!git pull origin master

From https://github.com/pschaldenbrand/Frida
 * branch            master     -> FETCH_HEAD
Already up to date.


In [ ]:
# !python -m pip install -r ../requirements.txt
!python -m pip install -r ../requirements_python3.txt

In [6]:
!mkdir painting

In [7]:
import requests
import PIL
from io import BytesIO
def save_image_from_internet(url, fn):
    response = requests.get(url)
    img = PIL.Image.open(BytesIO(response.content))
    img = img.convert('RGB')
    img.save(fn)

# Paint :]

In [ ]:
#@title # Conceptually Paint a Reference Image
#@markdown Paint from a source image using feature space loss.
#@markdown Features are extracted from the last convolutional layer of CLIP.

#@markdown Provide the URL to an image you'd like to paint

image = "https://cdn.homeandmoney.com/wp-content/uploads/2022/05/31113751/Pittsburgh_FeaturedImg-1.jpg" #@param {type:"string"}

!wget $image
image = image.split('/')[-1]

%tensorboard --logdir /content/Frida/src/painting
!python plan.py --simulate --max_height 256 --use_cache --cache_dir cache/ --n_stroke_models 1 \
        --objective clip_conv_loss \
        --objective_data $image  \
        --objective_weight 1.0 \
        --lr_multiplier 0.4 \
        --n_init 0 \
        --num_strokes 800 --optim_iter 400 \
        --n_colors 30

In [ ]:
#@title # Paint from Style and Text Input

#@markdown Paint based off of an example style (give a URL) and a language description of the painting

style_image = "https://media.newyorker.com/photos/5a82035156b75c08a3e5b5b4/4:3/w_2043,h_1532,c_limit/Cunningham-Obama-Kehinde-Wiley.jpg" #@param {type:"string"}
text_description = "A portrait of Samuel L Jackson smiling" #@param {type:"string"}

save_image_from_internet(style_image, 'input0.png')

%tensorboard --logdir /content/Frida/src/painting
!python plan.py --simulate --max_height 360 --use_cache --cache_dir cache/ --n_stroke_models 1 \
        --init_objective l2 --init_objective_data input0.png \
        --init_objective_weight 1.0 \
        --init_optim_iter  40 \
        --objective style text \
        --objective_data input0.png "$text_description" \
        --objective_weight 0.2 1.0 \
        --lr_multiplier 0.4 \
        --n_init 3 \
        --num_strokes 800 --optim_iter 300 \
        --n_colors 30

In [ ]:
#@title # Sketch2Painting with Style and Language Guidance

#@markdown Paint based off of a sketch to guide composition. Also give an example style and text description to help fill in details.

sketch = "https://as2.ftcdn.net/v2/jpg/02/75/70/77/500_F_275707791_2M7x7kOugTyRtsBCrrqpD1fYoLMMEkC7.jpg" #@param {type:"string"}
style_image = "https://uploads1.wikiart.org/images/norman-rockwell/ghostly-gourds.jpg!Large.jpg" #@param {type:"string"}
text_description = "Laura Dern flexin'" #@param {type:"string"}

save_image_from_internet(style_image, 'input0.png')

!wget $sketch
sketch = sketch.split('/')[-1]

%tensorboard --logdir /content/Frida/src/painting
!python plan.py --simulate --max_height 360 --use_cache --cache_dir cache/ --n_stroke_models 1 \
        --objective clip_conv_loss style text \
        --objective_data "$sketch" input0.png "$text_description" \
        --objective_weight 0.7 0.5 1.0 \
        --lr_multiplier 0.4 \
        --n_init 0 \
        --num_strokes 800 --optim_iter 300 \
        --n_colors 30

In [ ]:
#@title # Image Editing via Language

#@markdown Paint a given image with an edit that you describe

image = "https://vetstreet.brightspotcdn.com/dims4/default/ed822f6/2147483647/thumbnail/645x380/quality/90/?url=https%3A%2F%2Fvetstreet-brightspot.s3.amazonaws.com%2F75%2F371580ca7a11e0ad9e12313817c323%2Ffile%2FBeagle-1-645mk062311.jpg" #@param {type:"string"}
text_edit = "The dog is happy" #@param {type:"string"}

save_image_from_internet(image, 'input0.png')

%tensorboard --logdir /content/Frida/src/painting
!python plan.py --simulate --max_height 256 --use_cache --cache_dir cache/ --n_stroke_models 1 \
        --init_objective l2 --init_objective_data input0.png \
        --init_objective_weight 1.0 \
        --init_optim_iter  40 \
        --objective clip_fc_loss text \
        --objective_data input0.png "$text_edit" \
        --objective_weight 0.2 1.0 \
        --lr_multiplier 0.4 \
        --n_init 0 \
        --num_strokes 800 --optim_iter 300 \
        --n_colors 30

In [ ]:
#@title # Paint from an Image with Euclidean Distance
#@markdown Paint from a source image as exactly as possible.
#@markdown The algorithm tries to minimize the $L_2$ distance between the painting and the source iamge.

#@markdown Provide the URL to an image you'd like to paint

image = "https://cdn.homeandmoney.com/wp-content/uploads/2022/05/31113751/Pittsburgh_FeaturedImg-1.jpg" #@param {type:"string"}

save_image_from_internet(image, 'input0.png')

%tensorboard --logdir /content/Frida/src/painting
!python plan.py --simulate --max_height 256 --use_cache --cache_dir cache/ --n_stroke_models 1 \
        --objective l2 \
        --objective_data input0.png  \
        --objective_weight 1.0 \
        --lr_multiplier 0.4 \
        --n_init 0 \
        --num_strokes 800 --optim_iter 400 \
        --n_colors 30

In [ ]:
#@title # Draw with Ink
#@markdown Paint from a source image using feature space loss.
#@markdown Features are extracted from the last convolutional layer of CLIP.

#@markdown Provide the URL to an image you'd like to paint

image = "https://cdn.homeandmoney.com/wp-content/uploads/2022/05/31113751/Pittsburgh_FeaturedImg-1.jpg" #@param {type:"string"}

!wget $image
image = image.split('/')[-1]

%tensorboard --logdir /content/Frida/src/painting
!python plan.py --simulate --max_height 256 \
        --use_cache --cache_dir caches/cache_10_8_ink --n_stroke_models 1 \
        --objective clip_conv_loss \
        --objective_data $image  \
        --objective_weight 1.0 \
        --lr_multiplier 0.8 \
        --n_init 0 \
        --num_strokes 200 --optim_iter 400 \
        --ink

In [ ]:
 # Generalized Execution
 %tensorboard --logdir /content/Frida/src/painting --reload_interval 10 --reload_multifile True
!python plan.py --simulate --max_height 196 --use_cache --cache_dir cache/ --n_stroke_models 1 \
        --init_objective clip_conv_loss --init_objective_data frida.jpg --init_objective_weight 1.0 \
        --init_optim_iter  40 \
        --objective clip_conv_loss \
        --objective_data frida.jpg \
        --objective_weight 1.0 \
        --lr_multiplier 0.3 \
        --n_init 1\
        --num_strokes 800 --optim_iter 800 \
        --n_colors 12